In [104]:
import re
import numpy as np
import pandas as pd
import json
import glob

In [105]:
pd.set_option('display.max_rows', 300)
pd.set_option('display.max_columns', 50)

In [106]:
input_filespecs = [
    "../../../mnt/lab9-isilon/shared/eaglemonk2/examples-data-project-pvc-*/test-logs/*_longevity-test-*.log",
#     "../../../tmp/test-longevity-camera-claudio-04.log",
]

In [107]:
records = []
for input_filespec in input_filespecs:
    for input_filename in sorted(glob.glob(input_filespec)):
        print(input_filename)
        with open(input_filename) as file:
            for line in file:
                m = re.match('{.*', line)
                if m:
                    rec1 = json.loads(line)
                    rec = rec1['fields'] 
                    rec['level'] = rec1['level']
                    rec['target'] = rec1['target']
                    records += [rec]

../../../mnt/lab9-isilon/shared/eaglemonk2/examples-data-project-pvc-de7e9404-27e1-441d-8f6e-2732975871fb/test-logs/camera-claudio-04_longevity-test-claudio1-recorder-cc8b9f5f6-wfldt.log
../../../mnt/lab9-isilon/shared/eaglemonk2/examples-data-project-pvc-de7e9404-27e1-441d-8f6e-2732975871fb/test-logs/camera-claudio-04_longevity-test-claudio1-validator-0-6bffd48448-b24jf.log
../../../mnt/lab9-isilon/shared/eaglemonk2/examples-data-project-pvc-de7e9404-27e1-441d-8f6e-2732975871fb/test-logs/claudio2_longevity-test-claudio2-recorder-5646f589dc-p8k76.log
../../../mnt/lab9-isilon/shared/eaglemonk2/examples-data-project-pvc-de7e9404-27e1-441d-8f6e-2732975871fb/test-logs/claudio2_longevity-test-claudio2-validator-0-f487ff56b-jzkkx.log
../../../mnt/lab9-isilon/shared/eaglemonk2/examples-data-project-pvc-de7e9404-27e1-441d-8f6e-2732975871fb/test-logs/sim1e-0_longevity-test-sim1-recorder-0-d75fbd8d4-wqlnd.log
../../../mnt/lab9-isilon/shared/eaglemonk2/examples-data-project-pvc-de7e9404-27e1-441d

In [108]:
df = pd.DataFrame(records)
if "pts" not in df:
    df["pts"] = np.nan
if "prev_pts" not in df:
    df["prev_pts"] = np.nan
if "time_delta" not in df:
    df["time_delta"] = np.nan
df["min_pts"] = pd.to_datetime(df["min_pts"], errors="coerce")
df["max_pts"] = pd.to_datetime(df["max_pts"], errors="coerce")
df["pts"] = pd.to_datetime(df["pts"], errors="coerce")
df["prev_pts"] = pd.to_datetime(df["prev_pts"], errors="coerce")
df["ms_from_prev"] = df.pts.diff() / pd.Timedelta(1, "millisecond")

In [109]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1529 entries, 0 to 1528
Data columns (total 26 columns):
 #   Column                   Non-Null Count  Dtype              
---  ------                   --------------  -----              
 0   message                  62 non-null     object             
 1   level                    1529 non-null   object             
 2   target                   1529 non-null   object             
 3   description              1467 non-null   object             
 4   time_delta               820 non-null    object             
 5   prev_pts                 820 non-null    datetime64[ns, UTC]
 6   pts                      891 non-null    datetime64[ns, UTC]
 7   offset                   891 non-null    float64            
 8   size                     891 non-null    float64            
 9   flags                    891 non-null    object             
 10  probe_name               1467 non-null   object             
 11  stream                   1467 

In [110]:
df.head(4)

,message,level,target,description,time_delta,prev_pts,pts,offset,size,flags,probe_name,stream,element,pad,byte_count,buffer_count,min_pts,max_pts,pts_range,pts_missing_count,pts_gap_too_large_count,pts_decreasing_count,discontinuity_count,corrupted_count,idle_count,ms_from_prev
0,main: BEGIN,INFO,longevity_test,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"RUST_LOG=longevity_test=info,warn",INFO,longevity_test,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,"GST_DEBUG=pravegasrc:FIXME,qtdemux:ERROR,WARN",INFO,longevity_test,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,"opts=Opts { pravega_controller_uri: ""tls://pra...",INFO,longevity_test,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [111]:
stats = df[df.description.isin(["statistics"])].groupby(["stream", "probe_name"]).last()

In [112]:
pd.Timestamp.utcnow() - stats[["min_pts", "max_pts"]]

min_pts  \
stream                     probe_name                               
examples/camera-claudio-04 1-pravegasrc 6 days 15:21:09.539466001   
                           2-demux         6 days 15:21:09.539466   
                           3-parse         6 days 15:21:09.539466   
                           4-decode        6 days 15:21:09.539466   
examples/claudio2          1-pravegasrc 6 days 15:21:09.486466001   
                           2-demux         6 days 15:21:09.486466   
                           3-parse         6 days 15:21:09.486466   
                           4-decode        6 days 15:21:09.486466   
examples/sim1e-0           1-pravegasrc 0 days 00:16:42.615413112   
                           2-demux         0 days 00:16:42.615466   
                           3-parse         0 days 00:16:42.615466   
                           4-decode     0 days 00:16:42.615413112   
examples/sim1e-1           1-pravegasrc 0 days 00:17:04.053337667   
                           2-demux         0 days 00:17:04.053466   
                           3-parse         0 days 00:17:04.053466   
                           4-decode     0 days 00:17:04.053337667   
examples/sim2e-0           1-pravegasrc 0 days 00:13:09.483103889   
                           2-demux      0 days 00:13:09.483132667   
                           3-parse      0 days 00:13:09.483132667   
                           4-decode     0 days 00:13:09.483103889   
examples/sim2e-1           1-pravegasrc 0 days 00:10:52.983102445   
                           2-demux      0 days 00:10:52.983132667   
                           3-parse      0 days 00:10:52.983132667   
                           4-decode     0 days 00:10:52.983102445   
examples/sim2e-2           1-pravegasrc 0 days 00:12:01.933749889   
                           2-demux      0 days 00:12:01.933799334   
                           3-parse      0 days 00:12:01.933799334   
                           4-decode     0 days 00:12:01.933749889   
examples/sim2e-3           1-pravegasrc 0 days 00:13:09.983103889   
                           2-demux      0 days 00:13:09.983132667   
                           3-parse      0 days 00:13:09.983132667   
                           4-decode     0 days 00:13:09.983103889   

                                                          max_pts  
stream                     probe_name                              
examples/camera-claudio-04 1-pravegasrc 5 days 10:19:41.842466001  
                           2-demux         5 days 10:19:41.842466  
                           3-parse         5 days 10:19:41.842466  
                           4-decode        5 days 10:19:42.092466  
examples/claudio2          1-pravegasrc 6 days 10:48:56.624466001  
                           2-demux         6 days 10:48:56.624466  
                           3-parse         6 days 10:48:56.624466  
                           4-decode        6 days 10:48:57.024466  
examples/sim1e-0           1-pravegasrc 0 days 00:00:23.782080333  
                           2-demux      0 days 00:00:23.782132667  
                           3-parse      0 days 00:00:23.782132667  
                           4-decode        0 days 00:00:23.815466  
examples/sim1e-1           1-pravegasrc 0 days 00:01:02.686667889  
                           2-demux      0 days 00:01:02.686799334  
                           3-parse      0 days 00:01:02.686799334  
                           4-decode     0 days 00:01:02.720132667  
examples/sim2e-0           1-pravegasrc 0 days 00:00:57.549768001  
                           2-demux      0 days 00:00:57.549799334  
                           3-parse      0 days 00:00:57.549799334  
                           4-decode        0 days 00:00:57.566466  
examples/sim2e-1           1-pravegasrc 0 days 00:01:02.533098889  
                           2-demux      0 days 00:01:02.533132667  
                           3-parse      0 days 00:01:02.533132667  
           

In [114]:
stats[(stats.pts_missing_count>0) | 
      (stats.pts_decreasing_count>0) | 
      (stats.corrupted_count>0) | 
      (stats.discontinuity_count>100) | 
      (stats.idle_count>5)
     ].T

message
level
target
description
time_delta
prev_pts
pts
offset
size
flags
element


In [82]:
df[df.description.isin(["statistics"])].groupby(["stream", "probe_name"]).last().T

stream                            examples/camera-claudio-04  \
probe_name                                      1-pravegasrc   
message                                                  NaN   
level                                                   INFO   
target                                        longevity_test   
description                                       statistics   
byte_count                                       3.68185e+07   
buffer_count                                           11056   
min_pts                  2021-06-16 04:17:03.923999999+00:00   
max_pts                  2021-06-16 04:26:12.676999999+00:00   
pts_range                                 +0:09:08.753000000   
pts_missing_count                                          0   
pts_gap_too_large_count                                    0   
pts_decreasing_count                                       0   
discontinuity_count                                        1   
corrupted_count                                            0   
idle_count                                                 0   
element                                           pravegasrc   
pad                                                      src   
time_delta                                               NaN   
prev_pts                                                 NaT   
pts                                                      NaT   
offset                                                   NaN   
size                                                     NaN   
flags                                                    NaN   
idle_duration_ms                                         NaN   
ms_from_prev                                             NaN   

stream                                                     \
probe_name                                        2-demux   
message                                               NaN   
level                                                INFO   
target                                     longevity_test   
description                                    statistics   
byte_count                                    3.53791e+07   
buffer_count                                        11056   
min_pts                  2021-06-16 04:17:03.924000+00:00   
max_pts                  2021-06-16 04:26:12.677000+00:00   
pts_range                              +0:09:08.753000000   
pts_missing_count                                       0   
pts_gap_too_large_count                                 0   
pts_decreasing_count                                    0   
discontinuity_count                                     1   
corrupted_count                                         0   
idle_count                                              0   
element                                         h264parse   
pad                                                  sink   
time_delta                                            NaN   
prev_pts                                              NaT   
pts                                                   NaT   
offset                                                NaN   
size                                                  NaN   
flags                                                 NaN   
idle_duration_ms                                      NaN   
ms_from_prev                                          NaN   

stream                                                     \
probe_name                                        3-parse   
message                                               NaN   
level                                                INFO   
target                                     longevity_test   
description                                    statistics   
byte_count                                    3.53965e+07   
buffer_count                                        11056   
min_pts                  2021-06-16 04:17:03.924000+00:00   
max_pts                  2021-06-16 04:26:12.677000+00:00   
pts_range    

In [83]:
df.groupby(["stream", "description"]).size()

stream                      description            
examples/camera-claudio-04  Gap in PTS is too large     4
                            statistics                 36
examples/claudio2           Gap in PTS is too large    60
                            discontinuity              28
                            statistics                 36
examples/sim1d-0            statistics                 36
examples/sim1d-1            statistics                 36
examples/sim2d-0            statistics                 72
examples/sim2d-1            idle                        8
                            statistics                 72
examples/sim2d-2            discontinuity               8
                            statistics                 72
examples/sim2d-3            statistics                 72
dtype: int64

In [200]:
df[df.level.isin(["WARN","DEBUG"])].groupby(["stream", "description", "probe_name"]).size()

stream                      description              probe_name  
examples/camera-claudio-04  Gap in PTS is too large  1-pravegasrc      63
                                                     2-demux           63
                                                     3-parse           63
                                                     4-decode          86
                            PTS is missing           3-parse         1002
                                                     4-decode         792
                            discontinuity            1-pravegasrc      55
                                                     4-decode          54
                            idle                     1-pravegasrc       3
                                                     2-demux            3
                                                     3-parse            3
                                                     4-decode           3
examples/claudio2           Gap in PTS is too 

In [44]:
df[df.description.isin(["Gap in PTS is too large"])][["time_delta", "probe_name"]].value_counts().sort_index()

time_delta          probe_name  
+0:00:01.001000000  1-pravegasrc    2
                    2-demux         2
                    3-parse         2
                    4-decode        2
+0:00:01.002000000  1-pravegasrc    2
                                   ..
+0:10:48.114418555  1-pravegasrc    2
+0:14:05.150000000  2-demux         2
                    3-parse         2
                    4-decode        2
+0:14:05.150047888  1-pravegasrc    2
Length: 3395, dtype: int64

In [45]:
df[df.description.isin(["PTS is missing"])][["pts", "time_delta", "probe_name"]]

,pts,time_delta,probe_name


In [46]:
df[df.description.isin(["PTS is decreasing"])][["pts", "time_delta", "probe_name"]]

,pts,time_delta,probe_name


In [ ]:
df.iloc[300:400][df.probe_name.isin(["1-pravegasrc"])][["pts", "ms_from_prev", "prev_pts", "time_delta", "probe_name", "flags", "size", "description"]]

In [ ]:
df2 = df[df.description.isin(["buffer"]) & df.probe_name.isin(["1-pravegasrc"])].copy()

In [ ]:
df2["ms_from_prev"] = df2.pts.diff() / pd.Timedelta(1, "millisecond")

In [ ]:
df2[["pts", "ms_from_prev", "element", "pad", "flags", "size", "description"]].head(200)

In [ ]:
df2.ms_from_prev.value_counts()

In [ ]:
df2 = df[df.description.isin(["buffer"]) & ((df.element=="pravegasrc") | ((df.element=="h264parse") & (df["pad"]=="sink")))].copy()

In [ ]:
df2["ms_from_prev"] = df2.pts.diff() / pd.Timedelta(1, "millisecond")

In [ ]:
df2[["pts", "ms_from_prev", "element", "pad", "flags", "size", "description"]].head(200)

In [ ]:
df2[df2.element=="h264parse"].ms_from_prev.value_counts()

In [ ]:
df2.ms_from_prev.value_counts()